In [1]:
from mmengine.utils import get_git_hash
from mmengine.utils.dl_utils import collect_env as collect_base_env

import mmdet


def collect_env():
    """Collect the information of the running environments."""
    env_info = collect_base_env()
    env_info['MMDetection'] = f'{mmdet.__version__}+{get_git_hash()[:7]}'
    return env_info


if __name__ == '__main__':
    for name, val in collect_env().items():
        print(f'{name}: {val}')

sys.platform: linux
Python: 3.7.10 (default, Feb 26 2021, 18:47:35) [GCC 7.3.0]
CUDA available: True
MUSA available: False
numpy_random_seed: 2147483648
GPU 0: NVIDIA GeForce RTX 4090
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 11.1, V11.1.105
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.9.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.2-Product Build 20210312 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.1.2 (Git Hash 98be7e8afa711dc9b66c8ff3504129cb82013cdb)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_8

In [10]:
import pandas as pd
import cv2
import numpy as np
import PIL.Image as Image
import os

In [11]:
import os.path as osp
import mmcv
from mmengine.fileio import dump, load
import pandas as pd

labels = ["Disease"]


In [12]:
categories = [{"id":labels.index(x), "name":x} for x in labels]
categories

[{'id': 0, 'name': 'Disease'}]

In [13]:
def xywh_to_xyxy(xywh):
    """
    Convert XYWH format (x,y center point and width, height) to XYXY format (x,y top left and x,y bottom right).
    :param xywh: [X, Y, W, H]
    :return: [X1, Y1, X2, Y2]
    """
    if np.array(xywh).ndim > 1 or len(xywh) > 4:
        raise ValueError('xywh format: [x1, y1, width, height]')
    x1 = xywh[0] - xywh[2] / 2
    y1 = xywh[1] - xywh[3] / 2
    x2 = xywh[0] + xywh[2] / 2
    y2 = xywh[1] + xywh[3] / 2
    return np.array([x1, y1, x2, y2])

In [16]:
def convert_xray_to_coco(ann_file, out_file, image_prefix):
    df = pd.read_csv(ann_file)
    
    obj_count = 0
    annotations = []
    images = []
    
    means = []
    stds = []
    
    for idx, row in df.iterrows():
        label = row["Label"]
        filename =  row["Image"]
        img_path = osp.join(image_prefix, filename)
        image = mmcv.imread(img_path)
        height, width = image.shape[:2]
        channels = cv2.split(image)
        for channel in channels:
            means.append(np.mean(channel))
            stds.append(np.std(channel))
        images.append(
            dict(id=idx, file_name=filename, height=height, width=width))
        x_min,y_min,x_max,y_max = xywh_to_xyxy(row[["x", "y", "w", "h"]])
        
        data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=0,
                bbox=[x_min, y_min, row["w"],row["h"]],
                area=row["w"] * row["h"],
                iscrowd=0)
        annotations.append(data_anno)
        obj_count += 1
    
    means = np.array(means).reshape(-1, 3)
    stds = np.array(stds).reshape(-1, 3)
    
    # Compute the overall mean and std for each channel
    mean_per_channel = np.mean(means, axis=0)
    std_per_channel = np.mean(stds, axis=0)
    
    print(mean_per_channel)
    print(std_per_channel)
    
    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=categories)
    dump(coco_format_json, out_file)
convert_xray_to_coco("/data/train.csv", "/data/train_one_class.json", "/data/images")
convert_xray_to_coco("/data/test.csv", "/data/test_one_class.json", "/data/images")

[120.47771557 120.47771557 120.47771557]
[59.45722038 59.45722038 59.45722038]
[123.20065048 123.20065048 123.20065048]
[57.97565126 57.97565126 57.97565126]
